In [1]:
import yaml
import sys
import os
import pandas as pd
sys.path.append('..')

from wildlife_datasets import analysis, datasets, loader
from data.wildlife_dataset import WildlifeDataModule

root = '/Users/amee/Documents/code/master-thesis/datasets/'


In [2]:
path_dolphins = os.path.join(root, 'NDD20')
dataset_dolphins = datasets.NDD20v2(path_dolphins)

In [3]:
df = dataset_dolphins.metadata
df

,image_id,identity,path,orientation,segmentation,species,out_of_focus
0,0,unknown,ABOVE/104.jpg,above,"[2648.0, 1434.0, 2659.0, 1360.0, 2673.0, 1200....",BND,NaN
1,1,unknown,ABOVE/1841.jpg,above,"[1641.0, 1966.0, 1834.0, 1908.0, 2181.0, 1803....",BND,NaN
2,2,unknown,ABOVE/2118.jpg,above,"[1869.0, 2128.0, 2095.0, 2131.0, 2139.0, 2123....",BND,NaN
3,3,unknown,ABOVE/1042.jpg,above,"[1979.0, 1743.0, 1847.0, 1699.0, 1773.0, 1638....",BND,NaN
4,4,unknown,ABOVE/1042.jpg,above,"[1966.0, 1707.0, 1891.0, 1627.0, 1850.0, 1566....",BND,NaN
...,...,...,...,...,...,...,...
6097,6097,25,BELOW/1789.jpg,below,"[453.0, 628.0, 520.0, 585.0, 552.0, 571.0, 569...",WBD,False
6098,6098,70,BELOW/810.jpg,below,"[338.0, 407.0, 369.0, 428.0, 398.0, 419.0, 430...",WBD,False
6099,6099,70,BELOW/1555.jpg,below,"[545.0, 371.0, 601.0, 400.0, 626.0, 401.0, 644...",WBD,False
6100,6100,70,BELOW/1266.jpg,below,"[775.0, 533.0, 778.0, 523.0, 783.0, 514.0, 801...",WBD,False


In [4]:
import numpy as np
import pandas as pd
from PIL import Image

# 1) Inline helper functions

def get_bbox(seg: list[float]) -> list[float]:
    pts = np.array(seg, dtype=float).reshape(-1, 2)
    x0, y0 = pts.min(axis=0)
    x1, y1 = pts.max(axis=0)
    return [x0, y0, x1 - x0, y1 - y0]

def get_image_size(path: str) -> tuple[int, int]:
    path = os.path.join(root, 'NDD20', path)
    with Image.open(path) as img:
        return img.size  # (width, height)

# 2) Compute bbox for each row
df['bbox'] = df['segmentation'].apply(get_bbox)

# 3) Compute width & height by expanding into two columns
df[['width', 'height']] = (
    df['path']
      .apply(get_image_size)
      .apply(pd.Series)  # turns each (w,h) into two columns
)

# -- now df has 'bbox', 'width', 'height' added


In [5]:
df

,image_id,identity,path,orientation,segmentation,species,out_of_focus,bbox,width,height
0,0,unknown,ABOVE/104.jpg,above,"[2648.0, 1434.0, 2659.0, 1360.0, 2673.0, 1200....",BND,NaN,"[2648.0, 820.0, 499.0, 614.0]",5184,3456
1,1,unknown,ABOVE/1841.jpg,above,"[1641.0, 1966.0, 1834.0, 1908.0, 2181.0, 1803....",BND,NaN,"[1641.0, 859.0, 2472.0, 1671.0]",5184,3456
2,2,unknown,ABOVE/2118.jpg,above,"[1869.0, 2128.0, 2095.0, 2131.0, 2139.0, 2123....",BND,NaN,"[1721.0, 1993.0, 597.0, 223.0]",5184,3456
3,3,unknown,ABOVE/1042.jpg,above,"[1979.0, 1743.0, 1847.0, 1699.0, 1773.0, 1638....",BND,NaN,"[1434.0, 1522.0, 587.0, 240.0]",5184,3456
4,4,unknown,ABOVE/1042.jpg,above,"[1966.0, 1707.0, 1891.0, 1627.0, 1850.0, 1566....",BND,NaN,"[1759.0, 1511.0, 218.0, 223.0]",5184,3456
...,...,...,...,...,...,...,...,...,...,...
6097,6097,25,BELOW/1789.jpg,below,"[453.0, 628.0, 520.0, 585.0, 552.0, 571.0, 569...",WBD,False,"[453.0, 384.0, 509.0, 364.0]",1920,1080
6098,6098,70,BELOW/810.jpg,below,"[338.0, 407.0, 369.0, 428.0, 398.0, 419.0, 430...",WBD,False,"[333.0, 344.0, 360.0, 144.0]",1920,1080
6099,6099,70,BELOW/1555.jpg,below,"[545.0, 371.0, 601.0, 400.0, 626.0, 401.0, 644...",WBD,False,"[545.0, 323.0, 289.0, 127.0]",1920,1080
6100,6100,70,BELOW/1266.jpg,below,"[775.0, 533.0, 778.0, 523.0, 783.0, 514.0, 801...",WBD,False,"[747.0, 357.0, 125.0, 176.0]",1920,1080


In [6]:
import numpy as np

# only convert those entries that are numpy arrays, leave others untouched
df['segmentation'] = df['segmentation'].apply(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else x
)
df['segmentation'] = df['segmentation'].apply(lambda x: str([x]))
df['bbox'] = df['bbox'].apply(lambda x: str(x))

df

,image_id,identity,path,orientation,segmentation,species,out_of_focus,bbox,width,height
0,0,unknown,ABOVE/104.jpg,above,"[[2648.0, 1434.0, 2659.0, 1360.0, 2673.0, 1200...",BND,NaN,"[2648.0, 820.0, 499.0, 614.0]",5184,3456
1,1,unknown,ABOVE/1841.jpg,above,"[[1641.0, 1966.0, 1834.0, 1908.0, 2181.0, 1803...",BND,NaN,"[1641.0, 859.0, 2472.0, 1671.0]",5184,3456
2,2,unknown,ABOVE/2118.jpg,above,"[[1869.0, 2128.0, 2095.0, 2131.0, 2139.0, 2123...",BND,NaN,"[1721.0, 1993.0, 597.0, 223.0]",5184,3456
3,3,unknown,ABOVE/1042.jpg,above,"[[1979.0, 1743.0, 1847.0, 1699.0, 1773.0, 1638...",BND,NaN,"[1434.0, 1522.0, 587.0, 240.0]",5184,3456
4,4,unknown,ABOVE/1042.jpg,above,"[[1966.0, 1707.0, 1891.0, 1627.0, 1850.0, 1566...",BND,NaN,"[1759.0, 1511.0, 218.0, 223.0]",5184,3456
...,...,...,...,...,...,...,...,...,...,...
6097,6097,25,BELOW/1789.jpg,below,"[[453.0, 628.0, 520.0, 585.0, 552.0, 571.0, 56...",WBD,False,"[453.0, 384.0, 509.0, 364.0]",1920,1080
6098,6098,70,BELOW/810.jpg,below,"[[338.0, 407.0, 369.0, 428.0, 398.0, 419.0, 43...",WBD,False,"[333.0, 344.0, 360.0, 144.0]",1920,1080
6099,6099,70,BELOW/1555.jpg,below,"[[545.0, 371.0, 601.0, 400.0, 626.0, 401.0, 64...",WBD,False,"[545.0, 323.0, 289.0, 127.0]",1920,1080
6100,6100,70,BELOW/1266.jpg,below,"[[775.0, 533.0, 778.0, 523.0, 783.0, 514.0, 80...",WBD,False,"[747.0, 357.0, 125.0, 176.0]",1920,1080


In [7]:
df.to_csv('/Users/amee/Documents/code/master-thesis/EagleID/dataset/dataframe/cache_dolphins.csv', index=False)